In [3]:
import os
import sys

current_dir =  os.getcwd()
#parent_dir = os.path.dirname(current_dir)
#sys.path.insert(0, parent_dir)

grandparent_dir = os.path.dirname(os.path.dirname(current_dir))
sys.path.insert(0, grandparent_dir)


import pandas as pd

In [54]:
gdp = pd.read_csv(f"{grandparent_dir}/data/raw/CAGDP1_2001_2021.csv", encoding='ISO-8859-1')

gdp = gdp.loc[:9533] #drop notes at the bottom

for column in gdp.columns[[0,1,3]]:
    gdp[column] = gdp[column].astype(str)

    print(column)

gdp = gdp.drop(gdp.columns[5], axis=1) #drop Industry Classification column, as empty


for column in gdp.columns([7:]):
    gdp[column] = gdp[column].astype(float)
    
# focus on chain-type GDP
mask = gdp['Description'].str.startswith('Chain')
gdp = gdp[mask]

#gdp['GeoFIPS'].astype(str)

display(gdp.columns)


display(gdp.head())

assert gdp['GeoFIPS'].is_unique
assert gdp['Description'].str.startswith('Chain').all()


GeoFIPS
GeoName
TableName


Index(['GeoFIPS', 'GeoName', 'Region', 'TableName', 'LineCode', 'Description',
       'Unit', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017',
       '2018', '2019', '2020', '2021'],
      dtype='object')

,GeoFIPS,GeoName,Region,TableName,LineCode,Description,Unit,2001,2002,2003,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
1,"""00000""",United States,,CAGDP1,2.0,Chain-type quantity indexes for real GDP,Quantity index,81.601,82.985,85.305,...,100.000,101.842,104.172,106.991,108.775,111.214,114.489,117.116,113.875,120.646
4,"""01000""",Alabama,5,CAGDP1,2.0,Chain-type quantity indexes for real GDP,Quantity index,82.883,84.769,87.260,...,100.000,101.123,100.339,101.104,102.662,104.084,105.880,107.497,105.620,110.956
7,"""01001""","Autauga, AL",5,CAGDP1,2.0,Chain-type quantity indexes for real GDP,Quantity index,59.839,61.996,63.508,...,100.000,93.060,93.155,99.931,104.090,99.798,100.854,97.233,96.115,94.638
10,"""01003""","Baldwin, AL",5,CAGDP1,2.0,Chain-type quantity indexes for real GDP,Quantity index,73.853,77.273,81.570,...,100.000,104.651,106.431,110.434,115.476,118.498,125.068,131.431,131.614,144.294
13,"""01005""","Barbour, AL",5,CAGDP1,2.0,Chain-type quantity indexes for real GDP,Quantity index,113.864,111.853,114.628,...,100.000,113.335,106.760,103.702,101.969,100.509,101.804,102.053,98.044,99.393
